---

## Dense Model v1

---

Multivariate usage of a Dense NN.

In [ ]:
from datetime import datetime as dt
from sklearn.metrics import mean_squared_error, mean_absolute_error
from keras.layers import Dense,MaxPooling1D
from keras.layers import Flatten,RepeatVector
from keras.models import Sequential
from keras.utils import plot_model
from keras.callbacks import ModelCheckpoint, TensorBoard, Callback, EarlyStopping

In [ ]:
class Model_Densev1():
  """
  """
  def __init__(self, window_size=30, label_window=1, num_labels=1, num_epochs=300, debug=False):
    self.WINDOW_SIZE = window_size
    self.LABEL_WINDOW = label_window
    self.NUM_LABELS = num_labels
    self.NUM_EPOCHS = num_epochs
    self.debug = debug

    self.MODEL_NAME = "Densev1"

    if self.debug:
      print(f'### Building Model{self.MODEL_NAME}::')

  def get_name(self):
    return self.MODEL_NAME

  def train(self, X_train=None, y_train=None, num_features=None, dataset=None):
    """
    Build and train the model.
    """
    early_stop = EarlyStopping(monitor = "loss", mode = "min", patience = 25)
    model = Sequential()
    model.add(MaxPooling1D(pool_size=2, input_shape=(self.WINDOW_SIZE, num_features)))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Flatten())
    model.add(Dense(self.NUM_LABELS*self.LABEL_WINDOW))
    if (self.LABEL_WINDOW > 1):
      # reshape as => [batch, out_steps, labels]
      model.add(Reshape([self.LABEL_WINDOW, self.NUM_LABELS]))

    model.compile(loss='mae', optimizer='adam')

    if (dataset is not None):
      self.model_hist = model.fit(dataset, epochs=self.NUM_EPOCHS, callbacks = [early_stop], verbose=(1 if self.debug else 0))
    else:
      self.model_hist = model.fit(X_train, y_train, epochs=self.NUM_EPOCHS, verbose=1, callbacks = [early_stop] )
    self.model = model
    return self.model_hist

  def get_model_name(self, serial=None):
    return f"{dt.today().strftime('%Y%m%d-%H%M')}-{self.MODEL_NAME}-{serial if serial is not None else '0'}.hdf5"

  def save_model(self, path, serial=None):
    """
    Save the current model under the given drive path.
    Timestamp the model name.
    """
    fname = f'{path}{self.get_model_name(serial)}'
    print(f'Saving model to: {fname}')
    return self.model.save(fname)

  def predict(self, X_in):
    return self.model.predict(X_in)